In [232]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import xml.etree.ElementTree as ET
import re
import numpy as np
import nltk
import torch
from torch import nn
import transformers

from transformers import AutoTokenizer, AutoModel
from utils import *
from transformers import AutoTokenizer, BertForNextSentencePrediction
import torch
from sklearn.model_selection import train_test_split
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# SuperDialseg

In [249]:
with open("datasets/SuperDialseg/segmentation_file_train.json") as f:
    super_dialseg_json = json.load(f)


In [251]:
acc_dialog = []
for message in super_dialseg_json["dial_data"]["superseg-v2"][123]["turns"]:
    # acc_dialog.append()
    # print(message["utterance"])
    print(message["segmentation_label"])

0
0
0
0
0
0
1
0
0
0
0
0
0


# Датасет Анны

In [2]:
fpath = r"datasets/dataset_v1.xml"
root = ET.parse(fpath).getroot()

ids = []
situations = {}


def namespace(element):
    m = re.match(r"\{.*\}", element)
    return m.group(0) if m else ""


for child in root:
    text = []
    for attr in child:
        for a in attr:
            if "Text" in a.tag:
                text.append(str(a.text))
    ns = namespace(a.tag)
    if ns not in situations:
        situations[ns] = []
        situations[ns].append(text)
    else:
        situations[ns].append(text)

    msg = [int(attr.attrib["id"]) for attr in child]
    ids.append((msg[0], msg[-1]))
    # situations.append(text)

ids = sorted(ids, key=lambda x: x[0])

In [3]:
def print_situation(sit):
    for num, message in enumerate(sit):
        print(str(num) + ":", message.replace("\n", ""))

In [4]:
print_situation(situations["{JobSearch}"][15])

0: Starting a new part time job today. Unfortunately for my FCC work it's PHP. On the bright side, they are going to pay me. Hope it goes well.
1: Pay is always good, @StevenKW :)
2: This is true :) @admhlt hopefully I learn a lot.
3: My company deals with C, Java, and C#. I don't work on that side but I'm exposed to APIs and programming terms a lot, so I kind of pick up stuff along the way. Approach PHP in that way and it'll be tangentially useful. :)
4: @StevenKW  Well you’ve got to pay the bills! It’s a shame you don’t get to do double duty using JS for the gig 
5: that’d be so ideal
6: there are lots of PHP jobs still kicking around, so once you know both you’ll be that much more employable 
7: @StevenKW If it's a new project, you should just start it in JS :P See if they notice haha
8: @figitalboy yeah I was really focusing on JS so I was surprised to get the gig. But yeah I was in school all 2014 with no income so yeah I need to pay some bills.
9: I hear you man. Congrats on find

# Ubuntu Dialogue Corpus

In [90]:
UDC = pd.read_csv("datasets/UDC/train.csv")

In [103]:
UDC["Context"][5].replace("__eou__", "").split("__eot__")

["OOo should be there and installed by default, yes  use UNKNOWN in cases where a bug component doesn't exist  uh? every British keyboard I've used is pc105  ",
 ' for me, orinoco_cs is loaded my the pcmcia layer  ',
 ' oh, you mean all of this is just plugdev?  ',
 ' the plugdev group and all admin tools expecting sudo to work are the first two major issues i can see  ',
 ' sourcing the bash completion script slows down starting a shell by a good second for me :(  ',
 ' there is that :/  ',
 ' interestingly, though, ntpdate runs before pcmcia  ',
 ' urgh :)  ',
 ' oh, this is the "we must install exactly the same set of packages on every system!" thing  I keep meaning to jump up and down on mdz and jdub\'s heads until they relent in the case of pcmcia-cs :)  ',
 ' surely that can be detected at install time :)  ',
 ' yes, and debian-installer does so, but we have explicitly overridden this by policy  which I think is wrong in the PCMCIA case  ',
 ' ditto  ',
 ' ']

In [97]:
UDC["Utterance"][0]

'yes. same binary packages. __eou__'

In [85]:
UDC_df = pd.read_csv("datasets/ubuntu_dialogue_corpus/dialogueText.csv")

In [88]:
UDC_df2 = pd.read_csv("datasets/ubuntu_dialogue_corpus/dialogueText_196.csv")

In [89]:
UDC_df2

,folder,dialogueID,date,from,to,text
0,301,1.tsv,2004-11-23T11:49:00.000Z,stuNNed,NaN,any ideas why java plugin takes so long to load?
1,301,1.tsv,2004-11-23T11:49:00.000Z,crimsun,stuNNed,java 1.4?
2,301,1.tsv,2004-11-23T11:49:00.000Z,stuNNed,crimsun,yes
3,301,1.tsv,2004-11-23T11:49:00.000Z,crimsun,stuNNed,java 1.5 loads _much_ faster
4,301,1.tsv,2004-11-23T11:50:00.000Z,stuNNed,crimsun,noneus: how can i get 1.5 is there a .deb some...
...,...,...,...,...,...,...
9212872,13,3676.tsv,2012-07-07T20:17:00.000Z,MonkeyDust,legolas,= arian
9212873,13,3676.tsv,2012-07-07T20:18:00.000Z,MonkeyDust,legolas,"observation and deduction, dear watson"
9212874,13,16586.tsv,2008-07-25T01:53:00.000Z,linuxfce,NaN,i am trying to install nvidia drivers from the...
9212875,13,16586.tsv,2008-07-25T01:53:00.000Z,linuxfce,NaN,how do i enter runlevel 3? when i try telinit ...


In [32]:
len(UDC_df)

1038324

In [86]:
UDC_df

,folder,dialogueID,date,from,to,text
0,3,126125.tsv,2008-04-23T14:55:00.000Z,bad_image,NaN,"Hello folks, please help me a bit with the fol..."
1,3,126125.tsv,2008-04-23T14:56:00.000Z,bad_image,NaN,Did I choose a bad channel? I ask because you ...
2,3,126125.tsv,2008-04-23T14:57:00.000Z,lordleemo,bad_image,the second sentence is better english and we...
3,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,Sock Puppe?t
4,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,WTF?
...,...,...,...,...,...,...
1038319,3,51506.tsv,2012-01-31T10:56:00.000Z,DJ,NaN,anyone on?
1038320,3,51506.tsv,2012-01-31T10:56:00.000Z,aeon-ltd,DJ,yes
1038321,3,99669.tsv,2008-11-16T20:11:00.000Z,KR-data,NaN,can I get a pastebin of someones menu.lst with...
1038322,3,99669.tsv,2008-11-16T20:12:00.000Z,outbackwifi,KR-data,http://pastebin.com/fe921690


In [67]:
udc_dialogs = [list(x["text"]) for _, x in UDC_df.groupby("dialogueID")]

In [68]:
len(udc_dialogs)

346108

In [74]:
udc_dialogs[14]

['bonsoir', 'nobody speak french here?', "I do, but there's also #ubuntu-fr"]

In [75]:
np.mean([len(x) for x in udc_dialogs])

3.0

# Freecodecamp

In [134]:
df_freecodecamp = pd.read_csv(
    "datasets/freecodecamp/freecodecamp_casual_chatroom_anon.csv"
)

/tmp/ipykernel_764520/1581645885.py:1: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_freecodecamp = pd.read_csv(


In [135]:
print_situation(df_freecodecamp["text"][:30])

0: no legumes either
1: That bullet proof coffee sounds insane.
2: That guy has huge eyes.
3: @54a47e0cdb8155e6700e486e It is. but it works. some people just can't handle the taste :P
4: They guy that came up with the idea is kind of a joke though...
5: that sounds like torture actually :)
6: the*
7: I might try it out for fun-just one bullet proof coffee that is.
8: @54a44bbbdb8155e6700e47de I agree. he's pretty extreme lol
9: he's like the Bear Grylls of diets
10: haha
11: I have zero intention of doing the whole diet bit of it, I just want the nommy creamy fatty coffee
12: and the energy
13: I can't help but laugh at my own joke/reference lol
14: Anyone near LA/Santa Monica, CA want to host me and my wife for a week or two? :D
15: haha
16: I would if I didn’t have my son and his family camping in my den
17: tho we aren’t that close to santa monica
18: How close is "not that close"? lol
19: hmm, 2 hour drive, but thats because the freeways are a nightmare :)
20: Here's one of the thi

# Daily dialog

In [5]:
from datasets import load_dataset

dataset = load_dataset("daily_dialog")

In [ ]:
df1

In [ ]:
pd.DataFrame(dataset["validation"]).drop(["act", "emotion"], axis=1)["dialog"]

# Constructing test dataset

In [229]:


# df1 = pd.DataFrame(dataset["train"]).drop(["act", "emotion"], axis=1)["dialog"]
# df2 = pd.DataFrame(dataset["validation"]).drop(["act", "emotion"], axis=1)["dialog"]
# df3 = pd.DataFrame(dataset["test"]).drop(["act", "emotion"], axis=1)["dialog"]

# df = pd.concat([df1, df2, df3]).reset_index(drop=True)

# df_train = df

dialogs = []

for dialog in situations.values():
    for situation in dialog:
        dialogs.append(situation)

df2 = pd.DataFrame({"dialog": dialogs})

# print(len(df))

# df = pd.concat([df, df2["dialog"]])
# df = df.reset_index(drop=True)

# print(len(df))

# df_train, df_test = train_test_split(df, train_size=0.6, random_state=42)
# df2 = pd.concat([df2["dialog"], df3])
df_test = df2["dialog"].reset_index(drop=True)

In [ ]:
df_test

In [339]:
def process_super_dialseg(dataset_dict):
    # dataset_dict = {"messages": [], "boundaries": [], "boundaries_mask": []}

    for dialog in super_dialseg_json["dial_data"]["superseg-v2"]:
        messages = []
        boundaries = []
        boundaries_mask = []
        
        for num, message in enumerate(dialog["turns"]):
            messages.append(message["utterance"])
            # print(message["utterance"])
            boundaries_mask.append(message["segmentation_label"])
            if message["segmentation_label"]:
                boundaries.append(num)

        dataset_dict["messages"].append(np.array(messages))
        dataset_dict["boundaries"].append(np.array(boundaries[:-1]))
        dataset_dict["boundaries_mask"].append(np.array(boundaries_mask[:-1]))


In [340]:
def generate_test_dataset():
    random_generator = np.random.default_rng(42)

    dataset_dict = {"messages": [], "boundaries": [], "boundaries_mask": []}

    # for i in range(100):
    #     n_samples = df_test.sample(5, random_state=random_generator)
    #     seg_boundaries = []
    #     acc_sum = 0
    #     for sample in n_samples[:-1]:
    #         seg_boundaries.append(acc_sum + len(sample) - 1)
    #         acc_sum += len(sample)
    #     acc_sum += len(n_samples.iloc[-1])

    #     seg_boundaries_mask = np.zeros(acc_sum - 1, dtype=np.int8)
    #     dataset_dict["messages"].append(np.concatenate(n_samples.to_numpy()))
    #     dataset_dict["boundaries"].append(seg_boundaries)
    #     for pos in seg_boundaries:
    #         seg_boundaries_mask[pos] = 1
    #     dataset_dict["boundaries_mask"].append(seg_boundaries_mask)

        
    process_super_dialseg(dataset_dict)

    return pd.DataFrame(dataset_dict)


test_df = generate_test_dataset()
test_df



test_df.to_csv(f"synthetic_{len(test_df)}.csv", index=False)

In [325]:
len(test_df["messages"][0])

14

In [326]:
len(test_df["boundaries_mask"][0])

14

In [309]:
test_df.iloc[1]["boundaries_mask"]

array([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0])

In [310]:
test_df.iloc[1]

messages           [Can you help me with planing the benefits for...
boundaries                                                    [4, 6]
boundaries_mask                    [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]
Name: 1, dtype: object

In [315]:
test_df.iloc[1]["messages"]

array(['Can you help me with planing the benefits for my survivors?',
       'Are you planning for the future of your survivors?',
       'Not right now.',
       'But I want to know if my family would get any help from Social Security after my passing.',
       'yes, Social Security can help your family if you have earned enough credits through your work.',
       'And what about my widow, what benefits would she receive?',
       "She can get reduced benefits as early as age 60 or full benefits at full retirement age. If she's 50 years old she could get the benefits if she's disabled and her disability started 7 years or earlier before your death.",
       'what if she remarries after my death?',
       'if she remarries after age 60, 50 if disabled, she would still qualify for the benefits on your social security records.',
       'And if she receives a pension not covered by Social Security?',
       "If it's a pension based on work then their Social Security benefits as a survivor

In [164]:
len(test_df)

100

In [160]:
len(test_df)

100

In [152]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [341]:
calculate_random_mean_windowdiff(test_df)

0.5979709845399418

In [31]:
# import gc

# gc.collect()
# torch.cuda.empty_cache()

In [15]:
# calculate_mean_windowdiff(
#     test_df[:40], scorer=scorer_all_mpnet, smoothing_strategy="depth"
# )

0it [00:00, ?it/s]

40it [00:52,  1.32s/it]


0.3401264047570903

# Training bert

In [117]:
df_train_messages = df_train.reset_index()

In [12]:
df_train_messages.iloc[1].dialog

['Oh , man . I had the best supper last night . My wife made a stir-fry and it was amazing ! ',
 ' I love stir fry crispy bitesize vegetables covered in a mixture of soy sauce and oyster sauce . Wilted greens and fresh bean sprouts . Throw in some onion and garlic and ginger ! Mmm ! Mmm ! It ’ s almost lunchtime . I would die for a plate of stir fry right now ! ',
 ' Well , you can keep the vegetables , I ’ ll take the meat . The stir fry my wife made was really hearty , with chunks of beef and slivers of bell peppers and onion ... ',
 ' What ? You call that a stir fry ? More meat than vegetables ? That ’ s the worst insult you could throw at a Chinese stir fry . What a disgrace to the wok she fried it in ! What you had is equivalent to a fajita without the wrap ! ']

In [102]:
# df_train_messages = df_train[df_train.apply(len) > 30].reset_index(drop=True)

In [223]:
df_train_messages_exploded = df_train.explode()
df_train_messages_exploded = df_train_messages_exploded[df_train_messages_exploded.apply(len) > 30].reset_index(drop=True)

In [225]:
df_train_messages_exploded[0]

' Why , this is the most interesting film ! '

In [228]:
df_train_messages_exploded[2]

'Oh , man . I had the best supper last night . My wife made a stir-fry and it was amazing ! '

In [135]:
import random
random.seed(42)

rnd_gen = np.random.default_rng(42)

message_pairs = []
labels = []
for dialog in df_train_messages.iterrows():
    messages = dialog[1].dialog
    for i in range(len(messages) - 1):
        message_1 = messages[i]
        message_2 = messages[i + 1]
        if len(message_1.split()) < 30 and len(message_2.split()) < 30:
            continue

    
        message_3 = random.choice(df_train_messages_exploded)
        while message_3 == message_2 or len(message_3) < 30:
            message_3 = random.choice(df_train_messages_exploded)

        message_pairs.append(((message_1, message_2), (message_1, message_3))) 

        labels.append((1, 0))

pos_samples_num = len(labels)

# for i in range(pos_samples_num):
#     dialog = df_train_messages .sample(1, random_state=rnd_gen)
#     message_1 = random.choice(dialog["dialog"].iloc[0])
#     dialog = df_train_messages .sample(1, random_state=rnd_gen)
#     message_2 = random.choice(dialog["dialog"].iloc[0])

#     if message_1 == message_2:
#         i -= 1
#         continue

#     message_pairs.append((message_1 , message_2)) 
#     labels.append(0)


print(len(labels))

7533


In [ ]:
message_pairs[:40]

In [136]:
df_to_save = pd.DataFrame({"message_pairs": message_pairs, "labels": labels})

In [137]:
df_to_save.message_pairs[4]

((' The whole-wheat loaf is 45 cents , the white loaves are 35 cents each , the baguette is 27 cents and the rolls are eight cents each . That makes two dollars fifty-seven cents . ',
  ' There you go . '),
 (' The whole-wheat loaf is 45 cents , the white loaves are 35 cents each , the baguette is 27 cents and the rolls are eight cents each . That makes two dollars fifty-seven cents . ',
  " It is very dull , and the production isn't very satisfactory , either . "))

In [138]:
df_to_save.to_csv("message_pairs_train3.csv")

In [21]:
import ast
df = pd.read_csv("message_pairs_train.csv", converters={"message_pairs": ast.literal_eval})

FileNotFoundError: [Errno 2] No such file or directory: 'message_pairs_train2.csv'

In [111]:
df_to_save.to_csv("message_pairs_train.csv")

In [51]:
# # !pip install peft
# import peft
# from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

# peft_config = LoraConfig(
#     task_type=TaskType.SEQ_CLS, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1, target_modules=["output.dense", "intermediate.dense"]
# )
# model = get_peft_model(model, peft_config)
# model.load_state_dict(torch.load("bert_v4.bin"))
# model.eval()

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForNextSentencePrediction(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (outp

In [358]:
# model.load_state_dict(torch.load("bert_v2.bin"))
# model.eval()

model = BertForNextSentencePrediction.from_pretrained("bert-base-uncased").to("cuda:0")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model.load_state_dict(torch.load("bert_v13.bin"))
model.eval()
;

''

In [354]:
def scorer_trained_bert(messages: list[str]) -> list[float]:
    scores = []
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    device = torch.device("cuda:0")
    model.eval()

    for i in range(len(messages) - 1):
        j = i + 1

        tokenized_input = {}
        if tokenizer:
            tokenized_input = tokenizer(
                messages[i], messages[j], max_length=200, truncation=True, padding="max_length", return_tensors="pt"
            ).to(device)

        with torch.no_grad():
            logits = model.forward(**tokenized_input).logits
            # logits2 = model2.forward(**tokenized_input).logits

        score = F.softmax(logits, dim=1)[0][0]
        # score2 = F.softmax(logits2, dim=1)[0][0]
        # if abs(score2 - score) > 0.4:
        #     print(messages[i])
        #     print(messages[j])
        #     print(score.item(), score2.item())
        #     print("===============")
        # scores.append(score.item())
        # print(score.item())
        # print(logits)
        # score = float(logits[0][0] > logits[0][1])
        # print(score)
        scores.append(score)

    model.train()
    return scores

In [355]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy="depth")

0it [00:00, ?it/s]

40it [00:21,  1.85it/s]


0.5230348124098125

In [359]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy="depth")

0it [00:00, ?it/s]

40it [00:22,  1.80it/s]


0.5230348124098125

In [357]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy="depth")

0it [00:00, ?it/s]

40it [00:21,  1.83it/s]


0.5154103535353535

In [347]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy="depth")

40it [00:22,  1.79it/s]


0.5176488095238095

In [134]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy="depth")

0it [00:00, ?it/s]

40it [00:53,  1.33s/it]


0.34152871569022814

In [61]:
calculate_mean_windowdiff(test_df[:100], scorer=scorer_trained_bert, smoothing_strategy="depth")

0it [00:00, ?it/s]

100it [01:39,  1.01it/s]


0.3062774652885638

In [ ]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert)

In [ ]:
sim_cse_tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/unsup-simcse-bert-base-uncased")
simccse = AutoModel.from_pretrained("princeton-nlp/unsup-simcse-bert-base-uncased")
simccse.eval()

In [41]:
out = sim_cse_tokenizer("test, hello", return_tensors="pt")
o = simccse(**out) 
torch.flatten(o.pooler_output)

tensor([[ 1.8224e-02, -1.5342e-01,  2.4753e-02, -2.4195e-01,  1.0743e-02,
         -1.2324e-01, -2.8514e-01,  3.0996e-01, -3.4264e-01, -7.2695e-02,
          2.2181e-01,  8.9207e-02,  1.2024e-01,  3.9563e-01,  9.5287e-02,
          2.9778e-01,  4.0870e-01, -1.5835e-01, -1.1901e-01,  1.9070e-01,
          2.1353e-01, -1.4015e-01, -3.6019e-01,  4.5661e-01, -6.0788e-02,
          2.2946e-01,  1.3560e-01,  2.7423e-02,  1.3670e-01,  2.0928e-01,
         -1.3890e-01,  1.7705e-03,  1.5460e-02, -1.9037e-01, -1.6040e-01,
         -4.9947e-02, -3.9888e-01, -3.4945e-01,  3.1839e-01,  9.3402e-02,
         -2.0689e-01, -2.9903e-01,  1.2119e-03, -1.4769e-02, -2.1322e-03,
          8.7020e-02, -2.4439e-01, -8.7038e-02,  7.6707e-04,  1.8775e-02,
          2.5070e-01, -4.8939e-01, -5.9470e-02,  1.9797e-01, -3.5841e-01,
         -5.5158e-02, -4.5313e-01,  9.9714e-02,  1.6465e-01,  2.5107e-01,
          2.1619e-01,  1.8364e-03, -1.4387e-01,  1.6342e-01,  2.0781e-01,
          7.1705e-02,  2.5070e-01, -2.

In [23]:
from torch.nn.functional import cosine_similarity

In [34]:
def scorer_sim_cse(messages: list[str]) -> list[float]:
    scores = []
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    device = torch.device("cuda:0")
    model.eval()

    h = []
    for message in messages:
        tokens = sim_cse_tokenizer(message, return_tensors="pt")
        output = simccse(**tokens)
        h.append(torch.flatten(output.pooler_output))

    for i in range(len(messages) - 1):
        j = i + 1

        tokenized_input = {}
        if tokenizer:
            tokenized_input = tokenizer(
                messages[i], messages[j], max_length=45, truncation=True, padding="max_length", return_tensors="pt"
            ).to(device)

        with torch.no_grad():
            logits = model.forward(**tokenized_input).logits

        cos_sim = cosine_similarity(h[i], h[j],  dim=0)
        score = F.softmax(logits, dim=1)[0][0]

        # print(h[i])
        # print(h[j])

        # print(score)
        scores.append((score.item() * 0.5 + 0.5 * cos_sim.item()))
        # print(score.item())
        # print(logits)
        # score = float(logits[0][0] > logits[0][1])
        # print(score)
        # scores.append(score)

    model.train()
    return scores

In [60]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_sim_cse, smoothing_strategy="depth")

0it [00:00, ?it/s]

40it [01:14,  1.87s/it]


0.2984428859102616

In [54]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_sim_cse, smoothing_strategy="depth")

0it [00:00, ?it/s]

40it [01:14,  1.87s/it]


0.2965193623779101

In [56]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_all_mpnet, smoothing_strategy="depth")

0it [00:00, ?it/s]

40it [00:20,  1.94it/s]


0.3401264047570903

In [45]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, thr=0.3)

0it [00:00, ?it/s]

40it [00:30,  1.31it/s]


0.41600840826684643

In [30]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert)

40it [00:30,  1.33it/s]


0.30692733444408804

In [ ]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert)

In [25]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy="depth")

0it [00:00, ?it/s]

40it [00:42,  1.07s/it]


0.34237948340789404

In [15]:
# not trained
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy="depth")

40it [00:32,  1.23it/s]


0.37644891407645986

In [29]:
# not trained
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy="depth")

40it [00:32,  1.23it/s]


0.3290268482839446

In [37]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy=None)

40it [00:30,  1.30it/s]


0.9616573796789396

In [35]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy=None)

0it [00:00, ?it/s]

40it [00:30,  1.30it/s]


0.4553471084299834

In [23]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy=None)

0it [00:00, ?it/s]

40it [00:32,  1.24it/s]


0.34237948340789404

In [21]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy=None)

0it [00:00, ?it/s]

40it [00:32,  1.24it/s]


0.5687826951299265

In [25]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy=None)

0it [00:00, ?it/s]

40it [00:32,  1.24it/s]


0.4553471084299834

In [374]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy="depth")

40it [00:25,  1.58it/s]


0.439999519903291

In [376]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy="depth")

40it [00:25,  1.57it/s]


0.3963929229019702

In [372]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy="depth")

40it [00:26,  1.50it/s]


0.3927847767476985

In [380]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy="depth")

40it [00:24,  1.61it/s]


0.42016731702311494

In [544]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy="depth")

40it [00:24,  1.63it/s]


0.439999519903291

In [542]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy="depth")

40it [00:25,  1.60it/s]


0.35825837425069373

# Visualisation

In [242]:
print(
    segment_dialog(situation_text, scorer=scorer_all_mpnet, smoothing_strategy="depth")[
        0
    ]
)
print(
    segment_dialog(situation_text, scorer=scorer_all_mpnet, smoothing_strategy="depth")[
        1
    ]
)
print(
    segment_dialog(situation_text, scorer=scorer_all_mpnet, smoothing_strategy=None)[0]
)
print(
    segment_dialog(situation_text, scorer=scorer_all_mpnet, smoothing_strategy=None)[1]
)

[]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]
[ 1  2  3  6  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29 30 32 33 34 35 36 37 38 39 40 41]
[1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1]


In [247]:
situation = test_df.loc[14]
situation_text = situation["messages"]
orig_boundaries_mask = situation["boundaries_mask"]
pred_boundaries_mask = segment_dialog(
    situation_text, scorer=scorer_all_mpnet, smoothing_strategy="depth"
)[1]

for num, message in enumerate(situation_text):
    print(message.replace("\n", ""))
    if num < len(orig_boundaries_mask) and orig_boundaries_mask[num]:
        print("======")

    if num < len(pred_boundaries_mask) and pred_boundaries_mask[num]:
        print("------")

Could I be of any help to you ? 
 I want to buy a skirt , but I have no idea which one to buy . 
 Do you like miniskirt ? It's popular among young girls . The pleated skirt also sells well . 
 I don't like to keep up with joneses . 
 What about the divided skirt ? 
 I think this suits me . Can I try it on ? 
Excuse me , ma ’ am ? 
 Yes . What can I do for you ? 
 I ’ m new here and I can ’ t seem to find the lunch room . 
 Oh , it ’ s right downstairs . I ’ m going there myself . Just follow me . 
My name is Mary , and I will be your server this evening . 
 Hi Mary . We are really looking forward to a great meal here . 
 Can I interest you in an appetizer to start out ? 
 I would love an appetizer . Are they listed in the menu ? 
 We have our daily appetizers listed on the board over there on the wall . 
 I am thinking about the popcorn shrimp . How is that ? 
 That would be a great choice ! 
 I'll trust your taste and take one order of that . 
 We have a special where you can order a 

In [ ]:
calculate_mean_windowdiff(test_df, model, tokenizer)